# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [147]:
import pandas as pd
import seaborn as sns
import patsy
from seaborn import load_dataset

import matplotlib.pyplot as plt

import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [119]:
df = pd.read_csv('previsao_de_renda.csv')

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [172]:
#1.

In [121]:
df.columns

Index(['Unnamed: 0', 'data_ref', 'index', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'mau', 'renda'],
      dtype='object')

In [122]:
X = df[['sexo', 'posse_de_veiculo','posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia']]
y = df['renda']
X = pd.get_dummies(X)
X['posse_de_veiculo'] = X['posse_de_veiculo'].replace({True: 1, False: 0})
X['posse_de_imovel'] = X['posse_de_imovel'].replace({True: 1, False: 0})
X.columns




Index(['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'sexo_F', 'sexo_M',
       'tipo_renda_Assalariado', 'tipo_renda_Bolsista',
       'tipo_renda_Empresário', 'tipo_renda_Pensionista',
       'tipo_renda_Servidor público', 'educacao_Primário',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Casado', 'estado_civil_Separado', 'estado_civil_Solteiro',
       'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Aluguel',
       'tipo_residencia_Casa', 'tipo_residencia_Com os pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental'],
      dtype='object')

In [123]:
x_columns = ['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'sexo_F', 'sexo_M',
       'tipo_renda_Assalariado', 'tipo_renda_Bolsista',
       'tipo_renda_Empresário', 'tipo_renda_Pensionista',
       'tipo_renda_Servidor público', 'educacao_Primário',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Casado', 'estado_civil_Separado', 'estado_civil_Solteiro',
       'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Aluguel',
       'tipo_residencia_Casa', 'tipo_residencia_Com os pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental']
renamed_columns = []
for name in x_columns:
    renamed_columns.append(name.replace(" ", "_"))

X.columns = renamed_columns
formula = "renda ~ " + " + ".join(renamed_columns)
formula

'renda ~ posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia + sexo_F + sexo_M + tipo_renda_Assalariado + tipo_renda_Bolsista + tipo_renda_Empresário + tipo_renda_Pensionista + tipo_renda_Servidor_público + educacao_Primário + educacao_Pós_graduação + educacao_Secundário + educacao_Superior_completo + educacao_Superior_incompleto + estado_civil_Casado + estado_civil_Separado + estado_civil_Solteiro + estado_civil_União + estado_civil_Viúvo + tipo_residencia_Aluguel + tipo_residencia_Casa + tipo_residencia_Com_os_pais + tipo_residencia_Comunitário + tipo_residencia_Estúdio + tipo_residencia_Governamental'

In [125]:
X['tempo_emprego'] = X['tempo_emprego'].fillna(0)
X.isna().sum()

posse_de_veiculo                 0
posse_de_imovel                  0
qtd_filhos                       0
idade                            0
tempo_emprego                    0
qt_pessoas_residencia            0
sexo_F                           0
sexo_M                           0
tipo_renda_Assalariado           0
tipo_renda_Bolsista              0
tipo_renda_Empresário            0
tipo_renda_Pensionista           0
tipo_renda_Servidor_público      0
educacao_Primário                0
educacao_Pós_graduação           0
educacao_Secundário              0
educacao_Superior_completo       0
educacao_Superior_incompleto     0
estado_civil_Casado              0
estado_civil_Separado            0
estado_civil_Solteiro            0
estado_civil_União               0
estado_civil_Viúvo               0
tipo_residencia_Aluguel          0
tipo_residencia_Casa             0
tipo_residencia_Com_os_pais      0
tipo_residencia_Comunitário      0
tipo_residencia_Estúdio          0
tipo_residencia_Gove

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [140]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11250 entries, 10099 to 9382
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   posse_de_veiculo               11250 non-null  int64  
 1   posse_de_imovel                11250 non-null  int64  
 2   qtd_filhos                     11250 non-null  int64  
 3   idade                          11250 non-null  int64  
 4   tempo_emprego                  11250 non-null  float64
 5   qt_pessoas_residencia          11250 non-null  float64
 6   sexo_F                         11250 non-null  uint8  
 7   sexo_M                         11250 non-null  uint8  
 8   tipo_renda_Assalariado         11250 non-null  uint8  
 9   tipo_renda_Bolsista            11250 non-null  uint8  
 10  tipo_renda_Empresário          11250 non-null  uint8  
 11  tipo_renda_Pensionista         11250 non-null  uint8  
 12  tipo_renda_Servidor_público    11250 non-nu

In [141]:
df_test = X_train.merge(y_train, left_index=True, right_index=True)


# 2.Ridge

In [150]:
modelo = smf.ols(formula, data=df_test)

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2s = []
#aics = []

for alpha in alphas:
    reg = modelo.fit_regularized(method='elastic_net',
                            refit=True,
                            L1_wt=0.01,
                            alpha=alpha)
    y_pred = reg.predict(X_test)
    r2s.append(r2_score(y_test, y_pred))
    #aics.append(reg.aic)

In [151]:
for i in range(len(alphas)):
    print(f"Para o alpha={alphas[i]}, o R² é {r2s[i]}")

Para o alpha=0, o R² é 0.21798051646263217
Para o alpha=0.001, o R² é 0.21798051646263217
Para o alpha=0.005, o R² é 0.21798051646263217
Para o alpha=0.01, o R² é 0.21798051646263217
Para o alpha=0.05, o R² é 0.21798051646263217
Para o alpha=0.1, o R² é 0.21798051646263217


In [144]:
# Não há diferença no R² para os alphas apresentados

# 3.Lasso

In [152]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2s = []
#aics = []

for alpha in alphas:
    reg = modelo.fit_regularized(method='elastic_net',
                            refit=True,
                            L1_wt=1,
                            alpha=alpha)
    y_pred = reg.predict(X_test)
    r2s.append(r2_score(y_test, y_pred))
    #aics.append(reg.aic)

In [153]:
for i in range(len(alphas)):
    print(f"Para o alpha={alphas[i]}, o R² é {r2s[i]}")

Para o alpha=0, o R² é 0.21798051646263217
Para o alpha=0.001, o R² é 0.21798051646263217
Para o alpha=0.005, o R² é 0.21798051646263217
Para o alpha=0.01, o R² é 0.21798051646263217
Para o alpha=0.05, o R² é 0.21798051646263217
Para o alpha=0.1, o R² é 0.21798051646263217


# 4.Stepwise

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo_M                         with p-value 1.01861e-159
#############
['tempo_emprego', 'sexo_M']
Add  tipo_renda_Assalariado         with p-value 3.16144e-64
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado']
Add  idade                          with p-value 1.17977e-25
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado', 'idade']
Add  educacao_Superior_completo     with p-value 6.36175e-15
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado', 'idade', 'educacao_Superior_completo']
Add  sexo_F                         with p-value 1.5132e-14
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado', 'idade', 'educacao_Superior_completo', 'sexo_F']
Add  posse_de_imovel                with p-value 9.38155e-09
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado', 'idade', 'educacao_Superior_completo', 'sexo_F', 'posse_de_imovel']
Ad

In [154]:
formula = "renda ~ " + " + ".join(variaveis)

modelo = smf.ols(formula, data=df_test)
reg = modelo.fit()

y_pred = reg.predict(X_test)
r2_score(y_test, y_pred)

0.20147655605279746

# 5.Comparação

### Comentário:
Nenhum dos modelos apresentou um melhor R2. Portanto, apenas com essa métrica seria difícil escolher o melhor.

# 6.Transformações

In [156]:
formula = "renda ~ " + " + ".join(variaveis)
formula


'renda ~ tempo_emprego + sexo_M + tipo_renda_Assalariado + idade + educacao_Superior_completo + sexo_F + posse_de_imovel + tipo_renda_Servidor_público + educacao_Pós_graduação + tipo_residencia_Casa + tipo_residencia_Aluguel + tipo_renda_Pensionista'

In [170]:
formula = 'renda ~ tempo_emprego + np.power(tempo_emprego, 2) + sexo_M + tipo_renda_Assalariado + np.log(idade) + educacao_Superior_completo + sexo_F + posse_de_imovel + tipo_renda_Servidor_público + educacao_Pós_graduação + tipo_residencia_Casa + tipo_residencia_Aluguel + tipo_renda_Pensionista'
formula

'renda ~ tempo_emprego + np.power(tempo_emprego, 2) + sexo_M + tipo_renda_Assalariado + np.log(idade) + educacao_Superior_completo + sexo_F + posse_de_imovel + tipo_renda_Servidor_público + educacao_Pós_graduação + tipo_residencia_Casa + tipo_residencia_Aluguel + tipo_renda_Pensionista'

In [171]:
modelo = smf.ols(formula, data=df_test)
reg = modelo.fit()

y_pred = reg.predict(X_test)
r2_score(y_test, y_pred)

0.2315562934459694

### Comentário:
Foi possível melhorar o R2 à partir da transformação das variáveis `tempo_emprego` e `idade`

# 7.Árvore de regressão

In [173]:
from sklearn.tree import DecisionTreeRegressor

In [224]:
r2s = []
ind_i = []
ind_j = []
for i in range(2, 20):
    for j in range(20, 60):
        regr_1 = DecisionTreeRegressor(max_depth=i, min_samples_leaf=j)
        regr_1.fit(X_train, y_train)
        ind_i.append(i)
        ind_j.append(j)
        r2s.append(regr_1.score(X_test, y_test))


df_r2 = pd.DataFrame({'R2':r2s, 'profundidade':ind_i, 'n_minimo':ind_j})
#df_r2.pivot(index='profundidade', columns='n_minimo', values='R2').max()
#sns.heatmap(df_r2.pivot(index='profundidade', columns='n_minimo', values='R2'))
df_r2['R2'].max()

0.221866423756645

In [225]:
r2s = []
ind_i = []
ind_j = []
for i in range(2, 20):
    for j in range(20, 60):
        regr_1 = DecisionTreeRegressor(max_depth=i, min_samples_leaf=j)
        regr_1.fit(X_train[variaveis], y_train)
        ind_i.append(i)
        ind_j.append(j)
        r2s.append(regr_1.score(X_test[variaveis], y_test))


df_r2 = pd.DataFrame({'R2':r2s, 'profundidade':ind_i, 'n_minimo':ind_j})

#df_r2.pivot(index='profundidade', columns='n_minimo', values='R2')
#sns.heatmap(df_r2.pivot(index='profundidade', columns='n_minimo', values='R2'))
df_r2['R2'].max()

0.2196223016698814

### Comentário:
O R² máximo encontrado com árvore de regressão ainda é menor que o modelo de regressão linear utilizando o método stepwise e fazendo transformações (item 6)